In [1]:
def read_file(filepath):
    
    with open (filepath) as f:
        str_text = f.read()
    
    return str_text
    

In [2]:
#read_file('moby_dick_four_chapters.txt')

In [3]:
import spacy

In [4]:
nlp = spacy.load('en_core_web_lg',disable=['parser', 'tagger','ner']) # Disabling so as it processes much faster

In [5]:
nlp.max_length = 1198623

In [6]:
def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [7]:
d = read_file('moby_dick_four_chapters.txt')

In [8]:
tokens = separate_punc(d)

D:\anaconda\envs\dataenv\lib\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [11]:
len(tokens)

11338

In [12]:
# we will pass 25 words and let neural net predict 26th word 

In [14]:
train_len = 25 + 1
text_sequences = []

for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    
    text_sequences.append(seq)

In [17]:
' '.join(text_sequences[0])

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on'

In [18]:
' '.join(text_sequences[1])

'me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore'

In [19]:
from keras.preprocessing.text import Tokenizer

In [20]:
tokenizer = Tokenizer()

In [21]:
tokenizer.fit_on_texts(text_sequences)

In [22]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [24]:
for i in sequences[0]:
    print(f"{i} :> {tokenizer.index_word[i]}")

956 :> call
14 :> me
263 :> ishmael
51 :> some
261 :> years
408 :> ago
87 :> never
219 :> mind
129 :> how
111 :> long
954 :> precisely
260 :> having
50 :> little
43 :> or
38 :> no
314 :> money
7 :> in
23 :> my
546 :> purse
3 :> and
150 :> nothing
259 :> particular
6 :> to
2713 :> interest
14 :> me
24 :> on


In [25]:
len(tokenizer.word_counts)

2718

In [26]:
import numpy as np

In [27]:
sequences = np.array(sequences)

In [28]:
sequences

array([[ 956,   14,  263, ..., 2713,   14,   24],
       [  14,  263,   51, ...,   14,   24,  957],
       [ 263,   51,  261, ...,   24,  957,    5],
       ...,
       [ 952,   12,  166, ...,  262,   53,    2],
       [  12,  166, 2712, ...,   53,    2, 2718],
       [ 166, 2712,    3, ...,    2, 2718,   26]])

In [33]:
# LSTM based model and train test split
vocabulary_size = len(tokenizer.word_counts)

In [34]:
from keras.utils import to_categorical

In [35]:
X = sequences[:,:-1]
y = sequences[:,-1]

In [36]:
y = to_categorical(y,num_classes=vocabulary_size + 1,)

In [37]:
seq_len = X.shape[1]

In [38]:
X.shape

(11312, 25)

In [39]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding    # Lstm to handle sequences , Embedding layer to handle Vocabulary

In [40]:
def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, 25, input_length=seq_len))
    model.add(LSTM(150, return_sequences=True))
    model.add(LSTM(150))
    model.add(Dense(150, activation='relu'))

    model.add(Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
    model.summary()
    
    return model

In [41]:
model = create_model(vocabulary_size+1, seq_len)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 25)            67975     
                                                                 
 lstm (LSTM)                 (None, 25, 150)           105600    
                                                                 
 lstm_1 (LSTM)               (None, 150)               180600    
                                                                 
 dense (Dense)               (None, 150)               22650     
                                                                 
 dense_1 (Dense)             (None, 2719)              410569    
                                                                 
Total params: 787,394
Trainable params: 787,394
Non-trainable params: 0
_________________________________________________________________


In [42]:
from pickle import dump,load

In [ ]:
model.fit(X, y, batch_size=128, epochs=300,verbose=1)

Epoch 1/300
89/89 [==============================] - 31s 263ms/step - loss: 6.8476 - accuracy: 0.0455
Epoch 2/300
89/89 [==============================] - 20s 228ms/step - loss: 6.3871 - accuracy: 0.0529
Epoch 3/300
89/89 [==============================] - 20s 230ms/step - loss: 6.3545 - accuracy: 0.0529
Epoch 4/300
89/89 [==============================] - 23s 261ms/step - loss: 6.2353 - accuracy: 0.0526
Epoch 5/300
89/89 [==============================] - 25s 275ms/step - loss: 6.1309 - accuracy: 0.0529
Epoch 6/300
89/89 [==============================] - 18s 204ms/step - loss: 6.0512 - accuracy: 0.0596
Epoch 7/300
89/89 [==============================] - 18s 198ms/step - loss: 5.9132 - accuracy: 0.0655
Epoch 8/300
89/89 [==============================] - 18s 197ms/step - loss: 5.8214 - accuracy: 0.0682
Epoch 9/300
89/89 [==============================] - 17s 196ms/step - loss: 5.7353 - accuracy: 0.0709
Epoch 10/300
89/89 [==============================] - 18s 199ms/step - loss: 5.651

89/89 [==============================] - 17s 186ms/step - loss: 2.9512 - accuracy: 0.2870
Epoch 81/300
89/89 [==============================] - 16s 185ms/step - loss: 2.9276 - accuracy: 0.2934
Epoch 82/300
89/89 [==============================] - 16s 185ms/step - loss: 2.9081 - accuracy: 0.2979
Epoch 83/300
89/89 [==============================] - 17s 188ms/step - loss: 2.8848 - accuracy: 0.2995
Epoch 84/300
89/89 [==============================] - 17s 186ms/step - loss: 2.8678 - accuracy: 0.2984
Epoch 85/300
89/89 [==============================] - 17s 187ms/step - loss: 2.8397 - accuracy: 0.3127
Epoch 86/300
89/89 [==============================] - 17s 196ms/step - loss: 2.8293 - accuracy: 0.3120
Epoch 87/300
89/89 [==============================] - 17s 192ms/step - loss: 2.8020 - accuracy: 0.3156
Epoch 88/300
89/89 [==============================] - 17s 194ms/step - loss: 2.7724 - accuracy: 0.3248
Epoch 89/300
89/89 [==============================] - 17s 196ms/step - loss: 2.7504 - 

89/89 [==============================] - 18s 197ms/step - loss: 1.7208 - accuracy: 0.5519
Epoch 160/300
89/89 [==============================] - 18s 197ms/step - loss: 1.7207 - accuracy: 0.5525
Epoch 161/300
89/89 [==============================] - 18s 199ms/step - loss: 1.7097 - accuracy: 0.5514
Epoch 162/300
89/89 [==============================] - 17s 196ms/step - loss: 1.6923 - accuracy: 0.5609
Epoch 163/300
89/89 [==============================] - 18s 201ms/step - loss: 1.6763 - accuracy: 0.5673
Epoch 164/300
89/89 [==============================] - 17s 195ms/step - loss: 1.6622 - accuracy: 0.5663
Epoch 165/300
89/89 [==============================] - 18s 202ms/step - loss: 1.6576 - accuracy: 0.5667
Epoch 166/300
89/89 [==============================] - 18s 199ms/step - loss: 1.6534 - accuracy: 0.5687
Epoch 167/300
89/89 [==============================] - 17s 195ms/step - loss: 1.9203 - accuracy: 0.5245
Epoch 168/300
89/89 [==============================] - 17s 194ms/step - loss: 